<a href="https://colab.research.google.com/github/markhayes0811/Stock/blob/main/AfternoonModeExcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guide/Definitions:

| Term                       | Meaning                                                                |
| -------------------------- | ---------------------------------------------------------------------- |
| **Training Set**           | Subset of historical data used to fit the model                        |
| **Testing Set**            | Subset held back to test model performance                             |
| **Actual Mean Return (%)** | Return from `y_test` during testing                                    |
| **Realized Return (%)**    | True return from live market prices, used **after** the market session |


🧠 What Is the Training Set?
The training set is the portion of your historical data that the machine learning model uses to learn patterns — in your case, how early-morning stock data relates to short-term returns.

🔍 In Your Notebook Context:
1. You fetch intraday stock data from 09:30 to 12:00.

2. After feature engineering (e.g. RSI, MACD, VWAP), you:
          
          in python:
          X_train, X_test, y_train, y_test = train_test_split(features, labels)
                      
This splits the data into:

X_train, y_train → training set (usually 80%)

X_test, y_test → testing set (usually 20%)


3. Your model (e.g. XGBoost) is trained on:

python
Copy
Edit
model.fit(X_train, y_train)

4. Then you evaluate on the test set (X_test) to see how well the model generalizes.                       





⚠️ Clarification:
The **Actual Mean Return (%)** in your morning notebook is based on y_test, not real future data.

The **Realized Return (%)** in your afternoon notebook is calculated from actual post-market price data — it’s what truly happened.



\NEHC, HCTI, GNLN, CGTL, CVAC, GTI, SOAR, CGTL, RADX

| Column                        | Description                                                                                      |
| ----------------------------- | ------------------------------------------------------------------------------------------------ |
| **Predicted Mean Return (%)** | The model’s predicted average return **(in %)** for a specific interval (e.g., 9:30–10:00).      |
| **Actual Mean Return (%)**    | The average return observed **in the training set**, not from real post-facto market movement.   |
| **Signal**                    | Model-generated trading suggestion: `Buy` if prediction > +0.2%, `Sell` if < -0.2%, else `Hold`. |
| **Ticker**                    | The stock ticker symbol the prediction was made for (e.g., `HCTI`, `GTI`, etc).                  |
| **Realized Return (%)**       | The **true return from actual price data**, computed by the afternoon script (market-realized).  |
| **Abs Error**                 | The absolute difference between predicted and realized returns. Measures accuracy of the model.  |


⚠️ Important Distinction
✅ Actual Mean Return (%) = what the model trained on and tried to match.

✅ Realized Return (%) = what really happened, used for performance validation in the afternoon session.

You ideally want:

Predicted Mean Return (%) ≈ Realized Return (%)

Abs Error as small as possible

          ┌────────────────────┐
          │ Raw Intraday Data │   (09:30–12:00 1-min bars)
          └────────┬───────────┘
                   │
                   ▼
          ┌────────────────────┐
          │ Feature Engineering│   (returns, RSI, MACD, VWAP, etc.)
          └────────┬───────────┘
                   │
                   ▼
          ┌────────────────────┐
          │ Interval Labeling  │   ('early', 'mid', 'late')
          └────────┬───────────┘
                   │
                   ▼
      ┌────────────────────────────┐
      │ Train-Test Split (80/20)   │
      │                            │
      │  ┌────────────┐ ┌────────┐ │
      │  │ X_train    │ │ X_test │ │  ➤ Features
      │  │ y_train    │ │ y_test │ │  ➤ Labels (block returns)
      └──┴────────────┴─┴────────┘
                   │
                   ▼
          ┌────────────────────┐
          │ Model Training     │   (XGBoost fit on X_train, y_train)
          └────────┬───────────┘
                   │
                   ▼
          ┌────────────────────┐
          │ Predict on X_test  │
          └────────┬───────────┘
                   │
                   ▼
        ┌──────────────────────────────┐
        │ Compare to y_test            │
        │ ➤ Actual Mean Return (%)     │
        │ ➤ Predicted Mean Return (%)  │
        └────────────┬─────────────────┘
                     │
                     ▼
        ┌────────────────────────────────────┐
        │ Save predictions to CSV (morning)  │
        └────────────────┬───────────────────┘
                         │
                         ▼
        ┌────────────────────────────────────┐
        │ Fetch new price data (afternoon)   │
        │ ➤ Compute Realized Return (%)      │
        │ ➤ Compute Abs Error                │
        └────────────────────────────────────┘



🔁 Key Feedback Loops:
Morning mode: Predicted vs Test Set (y_test)

Afternoon mode: Predicted vs Realized (actual market outcome)

# Afternoon Mode.1 - beta (use Mode.2)

In [ ]:
# ✅ MODE TOGGLE
mode = 'afternoon'  # Options: 'morning' or 'afternoon'

# 📦 Install dependencies
!pip install pandas matplotlib seaborn scikit-learn xgboost openpyxl --quiet

# 🔧 Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from IPython.display import display, HTML
import pytz
import os
from pathlib import Path
from xgboost import XGBRegressor

# ✅ Grid Search Toggle
use_grid_search = False

# 🗂️ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
save_dir = Path('/content/drive/MyDrive/stock_predictions')
save_dir.mkdir(parents=True, exist_ok=True)

# 🎯 Polygon API
API_KEY = 'H3nRWzRqnMqojU9y1gkbo1UqTbl2peqf'

# 📈 Fetch minute-level data
def fetch_minute_data(ticker, date):
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}"
    params = {"adjusted": "true", "sort": "asc", "limit": 50000, "apiKey": API_KEY}
    r = requests.get(url, params=params)
    if r.status_code != 200:
        print(f"❌ Error fetching data for {ticker}: {r.text}")
        return None
    df = pd.DataFrame(r.json().get("results", []))
    if df.empty:
        return None
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.rename(columns={"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"})
    return df[['open', 'high', 'low', 'close', 'volume']]

# 🔍 Evaluate Predictions vs Actuals
def evaluate_actual_vs_predicted(pred_path, tickers, date):
    full_df = pd.read_csv(pred_path)
    all_evals = []

    for ticker in tickers:
        df = fetch_minute_data(ticker, date)
        if df is None:
            continue

        df = df.between_time("09:30", "12:00").copy()
        df['returns'] = df['close'].pct_change()

        df['interval'] = pd.cut(
            df.index.hour * 60 + df.index.minute,
            bins=[0, 600, 630, 720],
            labels=['early (9:30–10:00)', 'mid (10:00–10:30)', 'late (10:30–12:00)']
        )

        actuals = df.groupby('interval', observed=True)['returns'].sum() * 100

        try:
            pred = full_df[full_df['Ticker'] == ticker].set_index('Interval')
            if 'Predicted Mean Return (%)' not in pred.columns:
                print(f"❌ Missing predicted values for {ticker}")
                continue
            pred['Realized Return (%)'] = actuals
            pred['Abs Error'] = (pred['Predicted Mean Return (%)'] - pred['Realized Return (%)']).abs()
            pred['Ticker'] = ticker
            all_evals.append(pred.reset_index())
        except Exception as e:
            print(f"⚠️ Could not evaluate {ticker}: {e}")

    return pd.concat(all_evals, ignore_index=True) if all_evals else pd.DataFrame()

# ✅ User Input
tickers = input("Enter tickers separated by commas: ").strip().upper().split(',')
tickers = [t.strip() for t in tickers if t.strip()]
date_input = input("Enter trading date (YYYY-MM-DD): ").strip()

# ✅ Predictions path (from morning mode)
pred_file = save_dir / f"combined_predictions_{date_input}.csv"

if mode == 'afternoon' and pred_file.exists():
    result_df = evaluate_actual_vs_predicted(pred_file, tickers, date_input)

    if not result_df.empty:
        # Save styled Excel report
        excel_path = save_dir / f"evaluated_predictions_{date_input}.xlsx"

        def color_val(val):
            color = "green" if val > 0 else "red" if val < 0 else "black"
            return f"color: {color}"

        styled = result_df.style.format({
            "Predicted Mean Return (%)": "{:+.2f}",
            "Realized Return (%)": "{:+.2f}",
            "Abs Error": "{:.2f}"
        }).applymap(color_val, subset=["Predicted Mean Return (%)", "Realized Return (%)"])

        styled.to_excel(excel_path, index=False, engine='openpyxl')
        print(f"✅ Evaluation saved to Excel: {excel_path}")
    else:
        print("⚠️ No evaluations computed.")
else:
    print("❌ Predictions CSV not found or mode is not 'afternoon'")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter tickers separated by commas: NEHC, HCTI, GNLN, CGTL, CVAC, GTI, SOAR, CGTL, RADX
Enter trading date (YYYY-MM-DD): 2025-06-12
⚠️ Could not evaluate NEHC: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate HCTI: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate GNLN: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate CGTL: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate CVAC: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate GTI: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate SOAR: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate CGTL: "None of ['Interval'] are in the columns"
⚠️ Could not evaluate RADX: "None of ['Interval'] are in the columns"
⚠️ No evaluations computed.


# # Afternoon Mode.2

In [ ]:
# ✅ MODE TOGGLE
mode = 'afternoon'  # Options: 'morning' or 'afternoon'

# 📦 Install dependencies
!pip install pandas matplotlib seaborn scikit-learn xgboost openpyxl --quiet

# 🔧 Imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from IPython.display import display, HTML
from pathlib import Path

# 🗂️ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
save_dir = Path('/content/drive/MyDrive/stock_predictions')
save_dir.mkdir(parents=True, exist_ok=True)

# 🎯 Polygon API
API_KEY = 'H3nRWzRqnMqojU9y1gkbo1UqTbl2peqf'

# 📈 Fetch minute-level data
def fetch_minute_data(ticker, date):
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}"
    params = {"adjusted": "true", "sort": "asc", "limit": 50000, "apiKey": API_KEY}
    r = requests.get(url, params=params)
    if r.status_code != 200:
        print(f"\u274c Error fetching data for {ticker}: {r.text}")
        return None
    df = pd.DataFrame(r.json().get("results", []))
    if df.empty:
        return None
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df.rename(columns={"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"})
    return df[['open', 'high', 'low', 'close', 'volume']]

# 🔍 Evaluate Predictions vs Actuals
def evaluate_actual_vs_predicted(pred_path, tickers, date):
    full_df = pd.read_csv(pred_path)
    all_evals = []

    for ticker in tickers:
        df = fetch_minute_data(ticker, date)
        if df is None:
            continue

        df = df.between_time("09:30", "12:00").copy()
        df['returns'] = df['close'].pct_change()

        df['Interval'] = pd.cut(
            df.index.hour * 60 + df.index.minute,
            bins=[0, 600, 630, 660],
            labels=['early (9:30–10:00)', 'mid (10:00–10:30)', 'late (10:30–11:00)']
        )

        actuals = df.groupby('Interval', observed=True)['returns'].sum() * 100

        try:
            # Ensure alignment by converting intervals to strings
            actuals.index = actuals.index.astype(str)
            pred = full_df[full_df['Ticker'] == ticker].copy()
            pred['Interval'] = pred['Interval'].astype(str)
            pred.set_index('Interval', inplace=True)

            # Join and calculate error
            pred = pred.join(actuals.rename("Realized Return (%)"), how='left')
            pred['Abs Error'] = (pred['Predicted Mean Return (%)'] - pred['Realized Return (%)']).abs()
            pred['Ticker'] = ticker

            all_evals.append(pred.reset_index())
        except Exception as e:
            print(f"⚠️ Could not evaluate {ticker}: {e}")

    return pd.concat(all_evals, ignore_index=True) if all_evals else pd.DataFrame()

# ✅ User Input
tickers = input("Enter tickers separated by commas: ").strip().upper().split(',')
tickers = [t.strip() for t in tickers if t.strip()]
date_input = input("Enter trading date (YYYY-MM-DD): ").strip()

# ✅ Predictions path (from morning mode)
pred_file = save_dir / f"combined_predictions_{date_input}.csv"

if mode == 'afternoon' and pred_file.exists():
    result_df = evaluate_actual_vs_predicted(pred_file, tickers, date_input)

    if not result_df.empty:
        excel_path = save_dir / f"evaluated_predictions_{date_input}.xlsx"

        def color_val(val):
            color = "green" if val > 0 else "red" if val < 0 else "black"
            return f"color: {color}"

        styled = result_df.style.format({
            "Predicted Mean Return (%)": "{:+.2f}",
            "Realized Return (%)": "{:+.2f}",
            "Abs Error": "{:.2f}"
        }).applymap(color_val, subset=["Predicted Mean Return (%)", "Realized Return (%)"])

        styled.to_excel(excel_path, index=False, engine='openpyxl')
        print(f"\u2705 Evaluation saved to Excel: {excel_path}")
    else:
        print("\u26a0\ufe0f No evaluations computed.")
else:
    print("\u274c Predictions CSV not found or mode is not 'afternoon'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Enter tickers separated by commas: SLRX, VERV, INEO, SAFX, BGSF, XTIA, TDTH, RBNE
Enter trading date (YYYY-MM-DD): 2025-06-17


<ipython-input-11-3985383034>:103: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  }).applymap(color_val, subset=["Predicted Mean Return (%)", "Realized Return (%)"])


✅ Evaluation saved to Excel: /content/drive/MyDrive/stock_predictions/evaluated_predictions_2025-06-17.xlsx
